# Librar

## Introducción

### Sintaxis / Cheatsheet


## Ejemplo práctico: Análisis del sentimiento usando redes neuronales

El análisis del sentimiento consiste en detectar si una frase o fragmento de texto expresa una emoción que se pueda considerar 'positiva' o 'negativa'. Vamos a utilizar una librería que incluye modelos de redes neuronales basados en *transformers*.

In [24]:
!pip install transformers datasets evaluate
!pip install requests
!pip install newspaper3k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 54.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=4ab59e24a3f0489e80cd32deafdf6725040e1b4940441439a5bca6565b5b3537
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3340 sha256=107b6d337177d09818f82aee303e1e90e49d745f061469bb43c37492f6cc6dda
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1

Existen redes ya preentrenadas, en particular para análisis del sentimiento. Devuelven la etiqueta del texto (positiva o negativa) y la certeza de la decisión (si no está seguro, el score será menor)

In [11]:
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I love it", "I hate it"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998799562454224},
 {'label': 'NEGATIVE', 'score': 0.9996398687362671}]

Vamos a utilizar una API para ver el sentimiento de una frase aleatoria. Para ello podemos usar la siguiente función, queue utiliza el paquete `requests` de Python:

In [1]:
import requests

def get_random_quote(api_url):
    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            # Aquí cogemos el contenido de la petición
            quote_data = response.json()

            # Si la respuesta tiene contenido, nos quedamos con el primer resultado
            if len(quote_data) > 0:
              return quote_data[0].get('content', 'No quote available')

            # Si no, lanzar un error
            raise Exception("No hay pregunta disponible!")
        else:
            print(f"Error: {response.status_code} - {response.text}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Replace 'your_api_url' with the actual URL of the API
api_url = 'https://api.quotable.io/quotes/random'
quote = get_random_quote(api_url)

print("Quote : ", quote)
sentiment_pipeline([quote])

Podemos obtener una lista de frases aleatorias, y ordenarlas según cualquier criterio, añadiendo parámetros a la petición:

In [21]:
def get_random_quotes(api_url, n):
  try:
    #
    params = {'limit': n}
    response = requests.get(api_url, params=params)
    if response.status_code == 200:
        # Aquí cogemos el contenido de la petición
        quote_data = response.json()

        # Si la respuesta tiene contenido, nos quedamos con el primer resultado
        if len(quote_data) > 0:
          return [ q.get('content', 'No quote available') for q in quote_data['results'] ]

        # Si no, lanzar un error
        raise Exception("No quote available!")
    else:
        print(f"Error: {response.status_code} - {response.text}")

  except Exception as e:
      print(f"An error occurred: {e}")

api_url = 'https://api.quotable.io/quotes/random'

quotes = get_random_quotes('https://api.quotable.io/quotes', 5)
sentiments = sentiment_pipeline(quotes)

for i in range(len(sentiments)):
  sentiment = sentiments[i]
  print(f"Quote '{quotes[i]}' has {sentiment['label']} sentiment with {sentiment['score'] * 100.0} % confidence")

quote_data:  {'count': 5, 'totalCount': 2127, 'page': 1, 'totalPages': 426, 'lastItemIndex': 5, 'results': [{'_id': 'as0ElEk5g7', 'author': 'Charles Dickens', 'content': 'No one is useless in this world who lightens the burdens of another.', 'tags': ['Generosity'], 'authorSlug': 'charles-dickens', 'length': 68, 'dateAdded': '2023-04-14', 'dateModified': '2023-04-14'}, {'_id': '_EC8bhSDQa', 'author': 'Charles Dickens', 'content': 'Minds, like bodies, will often fall into a pimpled, ill-conditioned state from mere excess of comfort.', 'tags': ['Weakness'], 'authorSlug': 'charles-dickens', 'length': 102, 'dateAdded': '2023-04-14', 'dateModified': '2023-04-14'}, {'_id': 'qF9iB6wrCQ', 'author': 'Charles Dickens', 'content': "Subdue your appetites, my dears, and you've conquered human nature.", 'tags': [], 'authorSlug': 'charles-dickens', 'length': 67, 'dateAdded': '2023-04-14', 'dateModified': '2023-04-14'}, {'_id': 'R1DmP1kYtW', 'author': 'Charles Dickens', 'content': 'Reflect on your pres

In [32]:
from newspaper import Article

first_article = Article(url="https://www.fox13now.com/news/local-news/plans-for-high-speed-rail-between-salt-lake-city-and-las-vegas-not-moving-forward")
first_article.download()
first_article.parse()
sentiments = sentiment_pipeline([first_article.text[:512]])
print(sentiments)

[{'label': 'NEGATIVE', 'score': 0.9994753003120422}]
